# Zermelo Fraenkel axioms
#### Author: Matteo Caorsi

## Introduction

Why is it so important to give an axomatic definition of set theory? Well, a very naif answer is to give enough rules to teach a computer how to dal with sets! And if a computer can deal with sets, it can also deal with numbers! And from numbers, most of mathematics can be implemented on a computer. And here I am not just referring to numerical mathemtics:mI really mean alsmost all other fields, liike algebra and topology!

Of course, you can argue that dealing with infinty may be a bit tricky: let's see what we can do!

## Scope

In this short notebook we show how it is possible to build abstract set theory ona a computer using the ZFC axioms.

The idea is that the axioms should a compelte set of rules to havea computer understand and operate with abstacts sets -- defined as objects.

We will also be able to remove some of these axioms and see what the consequences are.

## The definition of a set

Well, it's really not easy to define a set without incurring into recursing definitioons: on the dicitonary wou will probably find something like "a collection of element". Sure, but what is a collection?

Instead, here we will approach sets in a dfferent way: **sets are rigid rooted trees**.

### Subdefintions

 - **tree**: a connectd graph with no cycles
 - **rooted**: a rooted tree is a tree with a designated vertex called root
 - **rigid**: a tree is said to be rigid if it has a trivial automorphism group


## Examples

A rigid tree:

![rig](./rigid.png)

A non rigid tree, as there is a non-trivial automorphism that permutes $(g, h, i)$

![non_rig](./non_rigid.png)

Let's define the `class` set to be based on these statements

In [1]:
from typing import List, Union

class Set:
    """The abstract set, that a computer can compute with"""
    elements: Union[List, List['Set']]
    
    def __init__(self, elements: Union[List, List['Set']]) -> None:
        self.elements = elements

s = Set([])  # the set containing the empty set
s.elements

[]

## Building everything from the empty set: Separation and Replacement axioms

The empty set in our current defintion is `[]`. It is more of a placeholder than anything else. The empty set correspond to the **leaf nodes** of the rooted graphs.

However, we are assuming its exstence: this is an axiom then! In the Zermelo-Fraenkel list of axioms there is actually not an "empty set axiom", but rather there are two, the **Separation and Replacement axioms**. Given $\phi$ a rule (i.e. an expression of first order logic composed by the basic logical operations), the two axioms state that:
 - $\exists x\in a, \phi(x)$, where $a$ is a set, $\phi(x)$ is a set as well (separation)
 - $\phi(a)$ is a set (replacement)

For non-empty sets, the separation axiom is equivalent to the replacement axiom. Hence, the separation axiom is equivalent to the existence of the empty set.

Then, the set containing the empty set is `Set([])` and this corresponds to a rooted graph with one edge and 2 vertices (one of which is the root).
Out of this basic constructions we can build more elaborate sets:

In [2]:
from copy import copy


# correct example
s0 = copy(s)
s1 = Set([s])
s2 = Set([s0,s1])

# wrong example -- for now it is still working
wrong = Set([s, s])

wrong.elements

# cleaning up for next section
del Set, s, s0, s1, s2

## Extensionality axiom
The above set would correspond to a non-rigid graph! Hence, we need to enhance the class `Set` with equality and make sure that there are no repeated elements.
This is the main purpose of the extensionality axiiom.

In [3]:
from typing import Set


class MyFirstSet:
    """A better class for sets"""
    elements: Union[Set, Set['MySet']]
    
    def __init__(self, elements: Union[Set, Set['MySet']]) -> None:
        self.elements = elements
        self.max = len(self.elements)
        self.elements_iter = iter(self.elements)
        
    def __eq__(self, other: 'MySet') -> bool:
        """equality between sets!"""
        return self.elements == other.elements  # we exploit the equalirty between `Set` defined in Python typing
        
    def __hash__(self) -> int:
        if isinstance(self.elements, set):
            return hash(str(self.elements))
        return hash(self.elements)
        
    def __repr__(self) -> str:
        return str(self.elements)

    def __next__(self):
        return next(self.elements_iter)


We have just changesd the implementation of our class to something a bit more structured.

In particular:
 - The empty set is now `set()`
 - The extensionality axiom is implemented and the equality of hte elements is checked recursively also using `Set` of Python
 - `__repr__` is for nicer prints

In [4]:
empty = set()

s0 = MyFirstSet(empty)
print("s0:", s0)
s1 = MyFirstSet({s0})
print("s1:", s1)
s0bis = copy(s0)
s2 = MyFirstSet({s0bis, s1})
print("s2:", s2)

s0: set()
s1: {set()}
s2: {{set()}, set()}


In [5]:
s1bis = copy(s1)
wrong_s2 = MyFirstSet({s1, s1bis})  # this is wrongly declared
assert len(wrong_s2.elements) == 1
print("declared two elements:", wrong_s2, "... but actually we are getting only one!")

del s0, s1, s0bis, s1bis, s2

declared two elements: {{set()}} ... but actually we are getting only one!


## Union and Pairing axioms

The two **axioms of union and pairing** tell us how two sets can be merged together. The pairing tells us that, given two sets $a$ and $b$ once can form the set $\{a, b\}$. Union means that we can do the union of the elements. Here is the implementation:

In [6]:
class MySet:
    """A definitely better class for sets"""
    elements: Union[Set, Set['MySet']]
    
    def __init__(self, elements: Union[Set, Set['MySet']]) -> None:
        self.elements = elements
        self.max = len(self.elements)
        self.elements_iter = iter(self.elements)
        
    def __eq__(self, other: 'MySet') -> bool:
        """equality between sets!"""
        return self.elements == other.elements  # we exploit the equalirty between `Set` defined in Python typing
        
    def __hash__(self) -> int:
        if isinstance(self.elements, set):
            return hash(str(self.elements))
        return hash(self.elements)
        
    def __repr__(self) -> str:
        return str(self.elements)

    def __next__(self):
        return next(self.elements_iter)
    
    def __len__(self):
        """this method is the key to mapping set
        theory to the natural numbers!!"""
        return len(self.elements)
    
    def __iter__(self):
        return self
    
    def union(self, other):
        """Union operation"""
        new_set_elements = self.elements.union(other.elements)
        new_set = MySet(new_set_elements)
        return new_set
    
    def pairing(self, other):
        """Pairing operation"""
        new_set = MySet({self, other})
        return new_set

In [7]:
empty = set()

s0 = MySet(empty) 

s1 = MySet({s0})
s2 = MySet({s1})
print(s2, s1)
s3 = s2.pairing(s1)
print(s2, s1, "  -- pairing -->  ", s3)
s4 = s2.union(s1)
print(s2, s1, "  -- union -->  ", s4)

print(len(s0), len(s1), len(s2), len(s3), len(s4))

{{set()}} {set()}
{{set()}} {set()}   -- pairing -->   {{set()}, {{set()}}}
{{set()}} {set()}   -- union -->   {{set()}, set()}
0 1 1 2 2


## Foundation axiom

In the setting above, te foundation axiom states that the branches  of the tree cannot have infite depth. This is related to **preveting the Russel paradox**.
Indeed, in case you accept that a set $a$ is allowed to satisfy this property: $a \in a$, then the the Russel paradox kicks in.

This is automatically implemented in python, as there cannot be infinite recursion on the set elements.

## The infinity axiom

In our setting, this axiom states that is is possible to have a set (i.e. a tree) which is infiniteyl wide.

We can implement this in python with iterators!

In [8]:
from typing import Type

class InfinitySet(MySet):
    def __init__(self, maximum: int = 10, cls: Type[MySet] = MySet) -> None:
        empty = set()
        self.cls = cls
        self.previous = self.cls(empty)  # start with empty set
        self.previous_singleton = self.cls(empty)  # start with empty set
        super().__init__(self.previous)
        self.maximum = maximum
        self.n = 0
        
    #def __iter__(self) -> 'InfinitySet':
    #    self.n: int = 0
    #    return self
        
    def __next__(self) -> MySet:
        if self.n < self.maximum:
            self.n +=1
            new = self.cls({self.previous_singleton})
            self.previous_singleton = copy(new)
            new = new.union(self.previous)
            self.previous = new
            return new
        else:
            raise StopIteration

As you can see in the next cell, the Infinity set has been generated up to the set with three elements:

In [9]:
inf_set = InfinitySet(3)  # need to run through it to get the elements
list(inf_set)[-1]

{{set()}, set(), {{set()}}}

## Powerset axiom

We have not yet defined what a subset is! Hence, in order to naturally be able to take powersets (i.e. the set of all the subsets) it is either necessary to define subsets or to axiomatise the existenc of the Poweset. Zermelo and Fraenkel chose the second option.

In [27]:
from itertools import chain, combinations


class MyImprovedSet(MySet):
    def powerset(self) -> 'MyImporvedSet':
        """powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"""
        s = self.elements
        iterable_of_tuples = chain.from_iterable(combinations(MyImprovedSet(s), r) for r in range(len(s) + 1))
        fun = lambda x: MyImprovedSet(set(x))
        list_of_tuples = list(iterable_of_tuples)
        print(list_of_tuples)
        print(set(map(fun, list_of_tuples)))
        return MyImprovedSet(set(map(fun, list_of_tuples)))
    
    
empty = set()

s0 = MyImprovedSet(empty)
print("s0:", s0)
s1 = MyImprovedSet({s0})
print("s1:", s1)
s0bis = copy(s0)
s2 = MyImprovedSet({s0bis, s1})
print("s2:", s2)
assert len(s2) == 2
print(f"The powerset of {s2} is: ", s2.powerset())
assert len(s2.powerset()) == 4

s0: set()
s1: {set()}
s2: {{set()}, set()}
[(), ({set()},), (set(),), ({set()}, set())]
{{set()}, {{set()}, set()}, set(), {{set()}}}
The powerset of {{set()}, set()} is:  {{set()}, {{set()}, set()}, set(), {{set()}}}
[(), ({set()},), (set(),), ({set()}, set())]
{{set()}, {{set()}, set()}, set(), {{set()}}}


By the way, the construction we were starting above with `s0,s1,s2` is the beginning of the **Von Neumann hierarchy**. In short, we build a sequence of sets
$$V_{\alpha+1} = Powerset(V_{\alpha}), V_0 = \emptyset$$
Once we reach "infinity", we also define 
$$V_\omega = Union(V_\alpha) \forall \alpha$$
and continue the hierarchy with 
$$V_{\omega+1} = Powerset(V_\omega)$$

For your information, $V_\omega$ is the universe of rational numbers, $V_{\omega+1}$ is the universe of real numbers and so on... with increasing cardinality (aleph).

## Axiom of choice

We come now to the most controversial axiom of all set theory: **the axiom of choice**.

It short, it states that, given a list of non-empty subsets of a set, it is always possible to pick one element for each subset.

The big question (and the controversy) is: how do we pick the element!? In case of the existence of an ordering for xample, once can simply define the representative element by choosing the smallest (or largest) element in the subset -- hence no need for the axiom of choice. But in the more general case?

In python we clearly see how important this axiom is, as we would really like to write a `element` method that returns a single element... but indeed, how do we pick it?

In [31]:
class MyFinalSet(MyImprovedSet):
    def pick(self):
        return list(self.elements)[0]  # one possibility among infinitely many others
        
empty = set()

s0 = MyFinalSet(empty)
print("s0:", s0)
s1 = MyFinalSet({s0})
print("s1:", s1)
s0bis = copy(s0)
s2 = MyFinalSet({s0bis, s1})
print("s2:", s2)
print("An element of s2 (which is again a set): ", s2.pick())

s0: set()
s1: {set()}
s2: {{set()}, set()}
An element of s2 (which is again a set):  {set()}
